### Importando bibliotecas necessárias

In [8]:
import requests
from datetime import datetime
import pandas as pd
import pandas_profiling
import pprint
import json
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor
from elasticsearch import Elasticsearch
from elasticsearch import helpers
es = Elasticsearch(http_compress=True)

In [7]:
pp = pprint.PrettyPrinter(indent=4)
pd.set_option('display.max_colwidth',100)

## Processo de ETL

#### Função que recupera a lista de todos os deputados

In [15]:
def listaDeputados(url):
    headers = {'accept': 'application/json'}
    r = requests.get(url, headers=headers)
    return r.json()['dados']

#### Carrega lista de deputados em um dataframe pandas

In [16]:
deputados = pd.DataFrame(listaDeputados('https://dadosabertos.camara.leg.br/api/v2/deputados?ordem=ASC&ordenarPor=nome'))

In [36]:
deputados.head()

,id,idLegislatura,nome,siglaPartido,siglaUf,uri,uriPartido,urlFoto
0,204554,56,ABÍLIO SANTANA,PL,BA,https://dadosabertos.camara.leg.br/api/v2/deputados/204554,https://dadosabertos.camara.leg.br/api/v2/partidos/37906,https://www.camara.leg.br/internet/deputado/bandep/204554.jpg
1,204521,56,ABOU ANNI,PSL,SP,https://dadosabertos.camara.leg.br/api/v2/deputados/204521,https://dadosabertos.camara.leg.br/api/v2/partidos/36837,https://www.camara.leg.br/internet/deputado/bandep/204521.jpg
2,204379,56,ACÁCIO FAVACHO,PROS,AP,https://dadosabertos.camara.leg.br/api/v2/deputados/204379,https://dadosabertos.camara.leg.br/api/v2/partidos/36763,https://www.camara.leg.br/internet/deputado/bandep/204379.jpg
3,204560,56,ADOLFO VIANA,PSDB,BA,https://dadosabertos.camara.leg.br/api/v2/deputados/204560,https://dadosabertos.camara.leg.br/api/v2/partidos/36835,https://www.camara.leg.br/internet/deputado/bandep/204560.jpg
4,204528,56,ADRIANA VENTURA,NOVO,SP,https://dadosabertos.camara.leg.br/api/v2/deputados/204528,https://dadosabertos.camara.leg.br/api/v2/partidos/37901,https://www.camara.leg.br/internet/deputado/bandep/204528.jpg


#### Função que recupera para cada deputado todas as despesas do ano de 2019, 

In [46]:
despesas = {}
list_dataframes = []
list_deputados = deputados.head(n=25)
for index, row in list_deputados.iterrows():
    pagina = 1
    while True:
        #print(pagina)
        url = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{row['id']}/despesas?ano=2019&pagina={pagina}&itens=100&ordem=ASC&ordenarPor=dataDocumento"
        #print(url)
        headers = {'accept': 'application/json'}
        r = requests.get(url, headers=headers)
        if not r.json()['dados']:
            break
        desp_dep = pd.DataFrame(r.json()['dados'])
        #desp_dep[['dep_id','dep_nome','dep_partido','dep_uf']] = row[['id','nome','siglaPartido','siglaUf']]
        desp_dep['dep_id'] = row['id']
        desp_dep['dep_nome'] = row['nome']
        desp_dep['dep_partido'] = row['siglaPartido']
        desp_dep['dep_uf'] = row['siglaUf']
        #display(desp_dep.head())
        list_dataframes.append(desp_dep)
        #print(list_dataframes)
        #print(type(despesas[row['id']]))
        pagina = pagina + 1
        update_progress(index/(len(list_deputados)-1))
        #print(f"{index} - {len(list_deputados)-1}")
despesas = pd.concat(list_dataframes)

Progress: [####################################################################################################] 100.0%


In [ ]:
despesas.profile_report(style={'full_width':True})

### Limpeza dos dados coletados
#### Nessa etapa será realizada a limpeza e adequação do dados baseado em uma análise utilizando a biblioteca `pandas_profile` através do comando `despesas.profile_report(style={'full_width':True})`
- Remoção de colunas desnecessárias do DataFrame, colunas que não contém dados ou já possuem representatividade em outra coluna do DataFrame
- Adequação de campos Nulos, inserindo valores padrão nessas posições
- Conversão dos tipos de dados para numerico e data

**Excluindo colunas desnecessárias**

In [47]:
despesas.drop(columns=['ano','codLote','parcela','urlDocumento','valorLiquido'],inplace=True)

In [50]:
despesas.dtypes

cnpjCpfFornecedor            object
codDocumento                  int64
codTipoDocumento              int64
dataDocumento        datetime64[ns]
mes                           int64
nomeFornecedor               object
numDocumento                 object
numRessarcimento             object
tipoDespesa                  object
tipoDocumento                object
valorDocumento              float64
valorGlosa                  float64
dep_id                        int64
dep_nome                     object
dep_partido                  object
dep_uf                       object
dtype: object

In [16]:
despesas.to_excel('despesas.xlsx')

In [ ]:
despesas.groupby(by=['dep_id']).count()

In [41]:
'despesas.count()

ano                  697
cnpjCpfFornecedor    697
codDocumento         697
codLote              697
codTipoDocumento     697
dataDocumento        672
mes                  697
nomeFornecedor       697
numDocumento         697
numRessarcimento     697
parcela              697
tipoDespesa          697
tipoDocumento        697
urlDocumento         697
valorDocumento       697
valorGlosa           697
valorLiquido         697
dep_id               697
dep_nome             697
dep_partido          697
dep_uf               697
dtype: int64

#### Ajustando os tipos de dados

In [49]:
despesas['dataDocumento'] = despesas['dataDocumento'].apply(lambda x: pd.to_datetime(x) if not pd.isna(x) else datetime(1970,1,1,0,0))

In [51]:
despesas['dataDocumento']

0    1970-01-01
1    1970-01-01
2    1970-01-01
3    1970-01-01
4    1970-01-01
5    2019-02-01
6    2019-02-02
7    2019-02-03
8    2019-02-06
9    2019-02-07
10   2019-02-07
11   2019-02-07
12   2019-02-08
13   2019-02-12
14   2019-02-12
15   2019-02-12
16   2019-02-13
17   2019-02-13
18   2019-02-14
19   2019-02-14
20   2019-02-14
21   2019-02-14
22   2019-02-15
23   2019-02-16
24   2019-02-16
25   2019-02-18
26   2019-02-18
27   2019-02-18
28   2019-02-19
29   2019-02-19
        ...    
45   2019-06-03
46   2019-06-03
47   2019-06-03
48   2019-06-03
49   2019-06-04
50   2019-06-05
51   2019-06-05
52   2019-06-05
53   2019-06-05
54   2019-06-07
55   2019-06-07
56   2019-06-07
57   2019-06-07
58   2019-06-09
59   2019-06-11
60   2019-06-12
61   2019-06-12
62   2019-06-13
63   2019-06-14
64   2019-06-17
65   2019-06-17
66   2019-06-17
67   2019-06-17
68   2019-06-18
69   2019-06-19
70   2019-06-19
71   2019-06-19
72   2019-07-05
73   2019-07-08
74   2019-07-16
Name: dataDocumento, Len

In [36]:
despesas.groupby(by=["dataDocumento"])['tipoDespesa'].count()

dataDocumento
2019-02-01     2
2019-02-02     2
2019-02-03     1
2019-02-04     1
2019-02-05     4
2019-02-06     3
2019-02-07    11
2019-02-08     4
2019-02-09     6
2019-02-10     1
2019-02-11     2
2019-02-12     7
2019-02-13     9
2019-02-14    15
2019-02-15     2
2019-02-16     3
2019-02-17     3
2019-02-18    17
2019-02-19     6
2019-02-20    12
2019-02-21    11
2019-02-22     7
2019-02-23     2
2019-02-24     4
2019-02-25     6
2019-02-26    13
2019-02-27     4
2019-02-28     4
2019-03-01     6
2019-03-02     3
              ..
2019-05-26     1
2019-05-27     9
2019-05-28     7
2019-05-29     4
2019-05-30     3
2019-05-31     6
2019-06-01     4
2019-06-03     2
2019-06-04     5
2019-06-05     1
2019-06-06     4
2019-06-07     1
2019-06-08     1
2019-06-11     2
2019-06-12     4
2019-06-13     5
2019-06-14     1
2019-06-17     2
2019-06-18     3
2019-06-19     6
2019-06-25     1
2019-06-26     3
2019-06-27     2
2019-06-28     3
2019-06-29     1
2019-07-01     1
2019-07-02     2


In [29]:
despesas.head(n=15)

,ano,cnpjCpfFornecedor,codDocumento,codLote,codTipoDocumento,dataDocumento,mes,nomeFornecedor,numDocumento,numRessarcimento,...,tipoDespesa,tipoDocumento,urlDocumento,valorDocumento,valorGlosa,valorLiquido,dep_id,dep_nome,dep_partido,dep_uf
0,2019,00000000000006,0,0,0,NaT,2,RAMAL,6836040,0,...,TELEFONIA,Nota Fiscal,,75.66,0.0,75.66,204554,ABÍLIO SANTANA,PL,BA
1,2019,00000000000006,0,0,0,NaT,3,RAMAL,6858322,0,...,TELEFONIA,Nota Fiscal,,10.44,0.0,10.44,204554,ABÍLIO SANTANA,PL,BA
2,2019,00000000000006,0,0,0,NaT,4,RAMAL,6860198,0,...,TELEFONIA,Nota Fiscal,,14.33,0.0,14.33,204554,ABÍLIO SANTANA,PL,BA
3,2019,00000000000006,0,0,0,NaT,5,RAMAL,6860309,0,...,TELEFONIA,Nota Fiscal,,16.89,0.0,16.89,204554,ABÍLIO SANTANA,PL,BA
4,2019,00000000000006,0,0,0,NaT,6,RAMAL,6867342,0,...,TELEFONIA,Nota Fiscal,,22.07,0.0,22.07,204554,ABÍLIO SANTANA,PL,BA
5,2019,02575829000148,6759637,1566457,0,2019-02-01,2,AVIANCA BRASIL,2472443863483,,...,PASSAGENS AÉREAS,Nota Fiscal,,2231.97,0.0,2231.97,204554,ABÍLIO SANTANA,PL,BA
6,2019,10348318000979,6758921,1566200,4,2019-02-02,2,WINDSOR ADM. DE HOTEIS E SERVIÇOS LTDA,78750,,...,"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.",Nota Fiscal Eletrônica,,1495.00,0.0,1495.00,204554,ABÍLIO SANTANA,PL,BA
7,2019,02575829000148,6759639,1566457,0,2019-02-03,2,AVIANCA BRASIL,2472444244801,,...,PASSAGENS AÉREAS,Nota Fiscal,,765.38,0.0,765.38,204554,ABÍLIO SANTANA,PL,BA
8,2019,69797870120,6762195,1567360,1,2019-02-06,2,ALEXANDRE,SEM NUMERO,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,25.00,0.0,25.00,204554,ABÍLIO SANTANA,PL,BA
9,2019,01654521108,6762169,1567360,1,2019-02-07,2,WILKER,sem numero,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,20.00,0.0,20.00,204554,ABÍLIO SANTANA,PL,BA


In [73]:
despesas.loc[despesas['dep_nome'].str.contains('AB')][['tipoDespesa','dataDocumento','dep_nome','valorDocumento']].groupby(by=['dep_nome','tipoDespesa']).agg(['sum','count','mean'])

valorDocumento  \
                                                                                    sum   
dep_nome       tipoDespesa                                                                
ABOU ANNI      COMBUSTÍVEIS E LUBRIFICANTES.                                    2639.12   
               FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                       4877.76   
               LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES                     468.00   
               MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR       15119.48   
               PASSAGENS AÉREAS                                                10456.67   
               SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO                        1615.51   
               SERVIÇOS POSTAIS                                                  985.96   
               TELEFONIA                                                        2372.31   
ABÍLIO SANTANA COMBUSTÍVEIS E LUBRIFICANTES.                                    9054.99   
               CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.                   31000.00   
               DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                            23000.00   
               Emissão Bilhete Aéreo                                           25273.40   
               FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                       1134.83   
               HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.           1686.10   
               LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES                   40207.00   
               MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR        9610.53   
               PASSAGENS AÉREAS                                                 4992.74   
               SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO                         427.00   
               SERVIÇOS POSTAIS                                                  548.83   
               TELEFONIA                                                         139.39   

                                                                                \
                                                                         count   
dep_nome       tipoDespesa                                                       
ABOU ANNI      COMBUSTÍVEIS E LUBRIFICANTES.                                21   
               FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                  110   
               LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES                 1   
               MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR    16   
               PASSAGENS AÉREAS                                             19   
               SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO                    49   
               SERVIÇOS POSTAIS                                             17   
               TELEFONIA                                                    18   
ABÍLIO SANTANA COMBUSTÍVEIS E LUBRIFICANTES.                                54   
               CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.                 8   
               DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                          5   
               Emissão Bilhete Aéreo                                        24   
               FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                   18   
               HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.        2   
               LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES                 8   
               MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR    13   
               PASSAGENS AÉREAS                                              4   
               SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO                    11   
               SERVIÇOS POSTAIS                                              7   
               TELEFONIA                                                     5   

                                                                                       
    

In [72]:
despesas.loc[despesas['dep_nome'].str.contains('AB')]

,ano,cnpjCpfFornecedor,codDocumento,codLote,codTipoDocumento,dataDocumento,mes,nomeFornecedor,numDocumento,numRessarcimento,...,tipoDespesa,tipoDocumento,urlDocumento,valorDocumento,valorGlosa,valorLiquido,dep_id,dep_nome,dep_partido,dep_uf
0,2019,00000000000006,0,0,0,None,2,RAMAL,6836040,0,...,TELEFONIA,Nota Fiscal,,75.66,0.00,75.66,204554,ABÍLIO SANTANA,PL,BA
1,2019,00000000000006,0,0,0,None,3,RAMAL,6858322,0,...,TELEFONIA,Nota Fiscal,,10.44,0.00,10.44,204554,ABÍLIO SANTANA,PL,BA
2,2019,00000000000006,0,0,0,None,4,RAMAL,6860198,0,...,TELEFONIA,Nota Fiscal,,14.33,0.00,14.33,204554,ABÍLIO SANTANA,PL,BA
3,2019,00000000000006,0,0,0,None,5,RAMAL,6860309,0,...,TELEFONIA,Nota Fiscal,,16.89,0.00,16.89,204554,ABÍLIO SANTANA,PL,BA
4,2019,00000000000006,0,0,0,None,6,RAMAL,6867342,0,...,TELEFONIA,Nota Fiscal,,22.07,0.00,22.07,204554,ABÍLIO SANTANA,PL,BA
5,2019,02575829000148,6759637,1566457,0,2019-02-01,2,AVIANCA BRASIL,2472443863483,,...,PASSAGENS AÉREAS,Nota Fiscal,,2231.97,0.00,2231.97,204554,ABÍLIO SANTANA,PL,BA
6,2019,10348318000979,6758921,1566200,4,2019-02-02,2,WINDSOR ADM. DE HOTEIS E SERVIÇOS LTDA,78750,,...,"HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.",Nota Fiscal Eletrônica,,1495.00,0.00,1495.00,204554,ABÍLIO SANTANA,PL,BA
7,2019,02575829000148,6759639,1566457,0,2019-02-03,2,AVIANCA BRASIL,2472444244801,,...,PASSAGENS AÉREAS,Nota Fiscal,,765.38,0.00,765.38,204554,ABÍLIO SANTANA,PL,BA
8,2019,69797870120,6762195,1567360,1,2019-02-06,2,ALEXANDRE,SEM NUMERO,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,25.00,0.00,25.00,204554,ABÍLIO SANTANA,PL,BA
9,2019,01654521108,6762169,1567360,1,2019-02-07,2,WILKER,sem numero,,...,"SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO",Recibos/Outros,,20.00,0.00,20.00,204554,ABÍLIO SANTANA,PL,BA


In [42]:
despesas['tipoDespesa'].value_counts()

COMBUSTÍVEIS E LUBRIFICANTES.                                152
FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                   128
SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO                    101
Emissão Bilhete Aéreo                                         84
TELEFONIA                                                     60
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR     57
PASSAGENS AÉREAS                                              42
SERVIÇOS POSTAIS                                              32
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                          26
LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES                 19
CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.                 11
HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.         2
LOCAÇÃO OU FRETAMENTO DE AERONAVES                             1
ASSINATURA DE PUBLICAÇÕES                                      1
Name: tipoDespesa, dtype: int64

In [43]:
despesas.isna().sum()

ano                   0
cnpjCpfFornecedor     0
codDocumento          0
codLote               0
codTipoDocumento      0
dataDocumento        25
mes                   0
nomeFornecedor        0
numDocumento          0
numRessarcimento      0
parcela               0
tipoDespesa           0
tipoDocumento         0
urlDocumento          0
valorDocumento        0
valorGlosa            0
valorLiquido          0
dep_id                0
dep_nome              0
dep_partido           0
dep_uf                0
dtype: int64

In [37]:
despesas.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 716 entries, 0 to 1
Data columns (total 21 columns):
ano                  716 non-null int64
cnpjCpfFornecedor    716 non-null object
codDocumento         716 non-null int64
codLote              716 non-null int64
codTipoDocumento     716 non-null int64
dataDocumento        691 non-null object
mes                  716 non-null int64
nomeFornecedor       716 non-null object
numDocumento         716 non-null object
numRessarcimento     716 non-null object
parcela              716 non-null int64
tipoDespesa          716 non-null object
tipoDocumento        716 non-null object
urlDocumento         716 non-null object
valorDocumento       716 non-null float64
valorGlosa           716 non-null float64
valorLiquido         716 non-null float64
dep_id               716 non-null int64
dep_nome             716 non-null object
dep_partido          716 non-null object
dep_uf               716 non-null object
dtypes: float64(3), int64(7), object(11)

### Gráficos

### Testes

In [ ]:
deputados.loc[deputados['nome'].str.contains('Tabata.*', regex=True, case=False)]['uri']

In [ ]:
listaDeputados('https://dadosabertos.camara.leg.br/api/v2/deputados?ordem=ASC&ordenarPor=nome')

In [ ]:
despesas[(despesas['tipoDespesa'].str.contains('COMBUSTÍVEIS'))].groupby(by=['nomeFornecedor'])['valorDocumento'].count().sort_values(ascending=False)

In [ ]:
despesas[(despesas['tipoDespesa'].str.contains('COMBUSTÍVEIS'))].groupby(by=['nomeFornecedor'])['valorDocumento'].sum().sort_values(ascending=False)

In [ ]:
despesas.loc[(despesas['nomeFornecedor'].str.contains('CASCOL COMBUSTIVEIS'))]

In [ ]:
despesas[(despesas['cnpjCpfFornecedor']=="17895646000187")].head()

### Funções auxiliares

In [31]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 100
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)